In [1]:
import pandas as pd
import numpy as np

# Data Imports

In [2]:
import sys, os
import re
from tqdm.auto import tqdm
from nltk import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
DATA_DIR = '../data'
trump_csv_file = "hashtag_donaldtrump.csv"
biden_csv_file = "hashtag_joebiden.csv"
trump_df_init = pd.read_csv(os.path.join(DATA_DIR, trump_csv_file), lineterminator='\n')
biden_df_init = pd.read_csv(os.path.join(DATA_DIR, biden_csv_file), lineterminator='\n')

In [5]:
biden_df = biden_df_init[biden_df_init['country'] == 'United States of America'].copy()
trump_df = trump_df_init[trump_df_init['country'] == 'United States of America'].copy()

In [6]:
biden_df

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
2,2020-10-15 00:00:20,1.316529e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3.494182e+09,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
4,2020-10-15 00:00:22,1.316529e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032807e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132
6,2020-10-15 00:00:25,1.316529e+18,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,Twitter for iPhone,1.994033e+07,Change Illinois | Biden will increase taxes by...,changeillinois,"Illinois, home of Lincoln and Reagan, used to ...",...,1397.0,"Chicago, Illinois",41.875562,-87.624421,Chicago,United States of America,North America,Illinois,IL,2020-10-21 00:00:03.106963698
11,2020-10-15 00:00:57,1.316529e+18,"FBI Allegedly Obtained Hunter Biden Computer, ...",0.0,0.0,Twitter for Android,9.607387e+17,RLCompton,NewfoundStudio,"""There are things known and there are things u...",...,266.0,"Kentucky, USA",37.572603,-85.155141,NaN,United States of America,North America,Kentucky,KY,2020-10-21 00:00:05.696100113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776861,2020-11-08 23:58:09,1.325588e+18,#election #2020Elections #trump #biden https:/...,0.0,0.0,Twitter for iPhone,9.538012e+17,NY Gay and MAGA,NYMike912,Gay proud. not a liberal. I was always a Democ...,...,1092.0,"New York, USA",40.712728,-74.006015,New York,United States of America,North America,New York,NY,2020-11-09 18:32:45.608982
776862,2020-11-08 23:58:10,1.325588e+18,@GOP @GOPLeader @senatemajldr @LindseyGrahamSC...,0.0,0.0,Twitter for Android,2.975783e+09,OnCapeCodTime,CapeCodBluesArt,"Indigenous American w/Scottish, Irish, German ...",...,9.0,"Cape Cod, MA",41.798807,-69.996014,NaN,United States of America,North America,Massachusetts,MA,2020-11-09 18:32:45.973973
776865,2020-11-08 23:58:24,1.325589e+18,@FLOTUS I’m excited to have a FLOTUS whose vag...,0.0,0.0,Twitter for iPhone,5.545625e+07,Caroline Billinson,cbillinson,my love language is dismantling the patriarchy.,...,436.0,"Washington, DC",38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-11-09 18:32:45.841439
776870,2020-11-08 23:58:48,1.325589e+18,The man needs some help...#usa #biden\nWhen wi...,0.0,0.0,Twitter for Android,1.248047e+18,Dr J,DrJoeMcCarthy,Human. Free Thinker. Met Mandela. Personal. Fa...,...,311.0,Earth. 3rd Planet from Sun.,43.519630,-114.315320,NaN,United States of America,North America,Idaho,ID,2020-11-09 18:32:45.641087


In [7]:
trump_df

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121
5,2020-10-15 00:00:17,1.316529e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,Twitter for Android,1.138416e+09,Farris Flagg,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,...,2363.0,"Perris,California",33.782519,-117.228648,NaN,United States of America,North America,California,CA,2020-10-21 00:00:01.866082651
7,2020-10-15 00:00:18,1.316529e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,Twitter for iPhone,9.007611e+17,Stacey Gulledge 🇺🇸 Patriot ♥️ KAG 🙏 👮‍♀️♥️,sm_gulledge,"Patriot, Wife, “Shaken not Stirred” Mom of two...",...,766.0,"Ohio, USA",40.225357,-82.688140,NaN,United States of America,North America,Ohio,OH,2020-10-21 00:00:02.612515712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970893,2020-11-08 23:58:46,1.325589e+18,@DonaldJTrumpJr Being a #tRump you are intimat...,0.0,0.0,Twitter for iPad,1.237758e+18,U N Known,pirkka2,Truth will save the world - not tRump. 🌎 VOTE ...,...,29.0,"New Jersey, USA",40.075738,-74.404162,NaN,United States of America,North America,New Jersey,NJ,2020-11-09 17:47:55.744839
970895,2020-11-08 23:58:51,1.325589e+18,Have you know decency #Trump @GOP? Do you kno...,3.0,0.0,Twitter Web App,4.835231e+07,"Linda ""I Voted for Biden"" Kenney Baden",KenneyBaden,"TrialLawyer https://t.co/1C0kKESSUC, Played by...",...,3024.0,"New York, USA",40.712728,-74.006015,New York,United States of America,North America,New York,NY,2020-11-09 17:47:55.833725
970897,2020-11-08 23:58:56,1.325589e+18,LIONZ DEN PRESENTS TO YOU \n\n“THE WHITE OBAMA...,1.0,0.0,Twitter for Android,9.868225e+17,IUIC Minnesota,IUICMinnesota,NaN,...,674.0,"Minnesota, USA",45.989659,-94.611329,NaN,United States of America,North America,Minnesota,MN,2020-11-09 17:47:55.847439
970900,2020-11-08 23:59:05,1.325589e+18,#Trump #Patriots #MAGA2020 https://t.co/2A8Fnm...,0.0,0.0,Twitter for Android,1.296581e+18,🍿🍷••HellWorld••🍷🍿,HellWor09724785,NaN,...,11.0,Hell on Earth,34.233137,-102.410749,NaN,United States of America,North America,Texas,TX,2020-11-09 17:47:55.939901


In [12]:
biden_df.loc[:,'who'] = 'biden'
trump_df.loc[:,'who'] = 'trump'
data = pd.concat([biden_df, trump_df])

In [9]:
def _calculate_languages_ratios(text):
    languages_ratios = {}

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

def detect_language(text):
    ratios = _calculate_languages_ratios(text)

    most_rated_language = max(ratios, key=ratios.get)

    return most_rated_language

In [10]:
def filter_english_one_to_one(df, who):
    tweets = df.query(f'who == "{who}" and state_code == state_code').sort_values('user_followers_count',ascending=False).drop_duplicates(['user_name'])[['tweet','long','lat','state_code']]
    languages = [detect_language(val) for val in tqdm(tweets.tweet)]
    filtered = tweets.loc[np.array(languages) == 'english']
    return filtered

In [13]:
filt_biden_df = filter_english_one_to_one(data, 'biden')
filt_trump_df = filter_english_one_to_one(data, 'trump')

In [14]:
def tweet_clean_up(df):
    remove_links = lambda txt: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', txt)
    remove_special = lambda txt: re.sub(r'[^a-zA-Z0-9!,.\'\"\?\-\s]', '', txt)
    remove_newline = lambda txt: re.sub(r'\n', ' ', txt)
    df['tweet'] = df['tweet'].apply(lambda x: remove_newline(remove_special(remove_links(x))))
    return df

In [15]:
cleaned_biden = tweet_clean_up(filt_biden_df)
cleaned_trump = tweet_clean_up(filt_trump_df)

In [16]:
cleaned_biden.loc[:,'sentiment'] = 0
cleaned_trump.loc[:,'sentiment'] = 0

In [17]:
cleaned_biden.to_csv(os.path.join(DATA_DIR, 'cleaned_1_to_1_biden.csv'), index=False, header=True)
cleaned_trump.to_csv(os.path.join(DATA_DIR, 'cleaned_1_to_1_trump.csv'), index=False, header=True)